In [7]:
from IPython.display import DisplayObject
import base64

class ESModule(DisplayObject):
    _MIME_TYPE = 'application/vnd.jupyter.esm-rich-output'
    
    def __init__(self, module=None, url=None):
        if module and url:
            raise ValueError('Cannot specify both module and url')
        if not module and not url:
            raise ValueError('Must specify either module or url')
        if module:
            url = 'data:text/javascript;base64,' + base64.b64encode(module.encode('utf-8')).decode('utf-8') 
        self._url = url
        self.data = {}
    
    def _repr_mimebundle_(self, include=None, exclude=None):
        mime_bundle = {} 
        mime_bundle[ExtensibleModule._MIME_TYPE] = self._url
        for key in self.data.keys():
            mime_bundle[key] = self.data[key]
        return mime_bundle

In [9]:
ESModule(module='''
export function render(output, element) {
    const div = document.createElement('div');
    div.style.background = 'lime';
    div.textContent = 'passed';
    element.appendChild(div);
}
''')

<__main__.ESModule object>

In [14]:

from IPython.display import Javascript
display(Javascript('window.nextOutputRendered = false'))
display(ESModule(module='''
export async function render(output, element) {
    const promise = new Promise((resolve) => setTimeout(resolve, 1000));
    await promise;
    if (window.nextOutputRendered) {
        const div = document.createElement('div');
        div.style.background = 'red';
        div.textContent = 'failed: expected subsequent outputs to wait for this.';
        element.appendChild(div);
    } else {
        const div = document.createElement('div');
        div.style.background = '#88FF88';
        div.textContent = 'passed';
        element.appendChild(div);
    }
}
'''))
display(Javascript('window.nextOutputRendered = true'))

<IPython.core.display.Javascript object>

<__main__.ESModule object>

<IPython.core.display.Javascript object>

In [15]:
data = ESModule(module='''
export function render(output, element) {
    const plainText = output.data['text/plain'];
    if (plainText != 'the text value') {
        const div = document.createElement('div');
        div.style.background = 'red';
        div.textContent = `Expected the text to be "the text value" but was ${plainText}`;
        element.appendChild(div);
    } else {
        const div = document.createElement('div');
        div.style.background = 'lime';
        div.textContent = 'passed';
        element.appendChild(div);
    }
}
''')
data.data['text/plain'] = 'the text value'
data

the text value